In [47]:
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import RandomizedSearchCV

Cargar datos en dataframe

In [48]:
data = pd.DataFrame()
data = pd.read_csv('winequality-red.csv')

In [49]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


Utilizamos la función de PCA para reducir el número de dimensiones(En este caso reducimos 11 dimensiones a 2)

In [51]:
# Separar las características de la variable objetivo
X = data.drop('quality', axis=1)
print(f"Dimensiones originales: {X.shape[1]}")

# Aplicar PCA
pca = PCA(n_components=0.95)
X = pca.fit_transform(X)

print(f"Dimensiones reducidas: {X.shape[1]}")

Dimensiones originales: 11
Dimensiones reducidas: 2


Mostramos los valores que quedaron despues de la redución de dimensiones

In [52]:
X = pd.DataFrame(X)
X

,0,1
0,-13.224905,-2.023900
1,22.037724,4.408322
2,7.162673,-2.501461
3,13.430063,-1.951122
4,-13.224905,-2.023900
...,...,...
1594,1.129111,16.313102
1595,9.489984,21.627525
1596,-3.431354,14.271243
1597,1.132878,16.316167


Probamos con los kernel(linear, sigmoid y rfb) para comprobar la precisión(Sin hipermarámetros) de los diferentes kernel con los datos con dimensiones reducidas

In [53]:
# Definir la variable objetivo
y = (data['quality'] >= 7).astype(int)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Probar con distintos kernels
kernels = ['linear', 'sigmoid', 'rbf']
for kernel in kernels:
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    
    print(f"Kernel: {kernel}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print("\n")

Kernel: linear
Accuracy: 0.853125
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       273
           1       0.00      0.00      0.00        47

    accuracy                           0.85       320
   macro avg       0.43      0.50      0.46       320
weighted avg       0.73      0.85      0.79       320



Kernel: sigmoid
Accuracy: 0.815625
              precision    recall  f1-score   support

           0       0.85      0.95      0.90       273
           1       0.17      0.06      0.09        47

    accuracy                           0.82       320
   macro avg       0.51      0.50      0.49       320
weighted avg       0.75      0.82      0.78       320



Kernel: rbf
Accuracy: 0.85625
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       273
           1       1.00      0.02      0.04        47

    accuracy                           0.86       320
   macro avg       0.93  

c:\Users\alejandro.regueiroru\AppData\Local\miniconda3\envs\MachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alejandro.regueiroru\AppData\Local\miniconda3\envs\MachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alejandro.regueiroru\AppData\Local\miniconda3\envs\MachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

Ahora vamos a probar con unos hiperparámetros para ver si mejoran los resultados

In [54]:

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 0.01],
    'kernel': ['linear', 'rbf', 'sigmoid']
}

# Crear una instancia de RandomizedSearchCV
random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=10, cv=5, random_state=42)

# Ajustar el modelo con los datos de entrenamiento
random_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {random_search.best_params_}")
print(f"Mejor precisión: {random_search.best_score_}")

Mejores hiperparámetros: {'kernel': 'linear', 'gamma': 0.1, 'C': 0.1}
Mejor precisión: 0.8670833333333334


Probamos ahora los mismos kernel pero con los datos originales

In [55]:
X = data.drop('quality', axis=1)
# Definir la variable objetivo
y = (data['quality'] >= 7).astype(int)

In [56]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [57]:


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Probar con distintos kernels
kernels = ['linear', 'sigmoid', 'rbf']
for kernel in kernels:
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    
    print(f"Kernel: {kernel}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print("\n")

Kernel: linear
Accuracy: 0.853125
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       273
           1       0.00      0.00      0.00        47

    accuracy                           0.85       320
   macro avg       0.43      0.50      0.46       320
weighted avg       0.73      0.85      0.79       320



Kernel: sigmoid
Accuracy: 0.853125
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       273
           1       0.00      0.00      0.00        47

    accuracy                           0.85       320
   macro avg       0.43      0.50      0.46       320
weighted avg       0.73      0.85      0.79       320



Kernel: rbf
Accuracy: 0.85625
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       273
           1       1.00      0.02      0.04        47

    accuracy                           0.86       320
   macro avg       0.93  

c:\Users\alejandro.regueiroru\AppData\Local\miniconda3\envs\MachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alejandro.regueiroru\AppData\Local\miniconda3\envs\MachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alejandro.regueiroru\AppData\Local\miniconda3\envs\MachineLearning\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

Probamos también los hiperparámetros con los datos originales

In [58]:
# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 0.01],
    'kernel': ['linear', 'rbf', 'sigmoid']
}

# Crear una instancia de RandomizedSearchCV
random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=10, cv=5, random_state=42)

# Ajustar el modelo con los datos de entrenamiento
random_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print(f"Mejores hiperparámetros: {random_search.best_params_}")
print(f"Mejor precisión: {random_search.best_score_}")

Mejores hiperparámetros: {'kernel': 'rbf', 'gamma': 0.01, 'C': 10}
Mejor precisión: 0.8686427696078433


### Podemos observar que usando los 3 mismos kernels, 2 de ellos presentan los mismos resultados, a excepción del kernel sigmoid que en los datos originales presenta un mejor resultado por una diferencia de un 4% aproximadamente 

### En el resultado dado usando algunos hiperparámetros, podemos ver que ofrece un mejor resultado usar los datos originales, pero la diferencia es muy pequeña 0.869 con los datos originales y 0.867 con los datos con PCA

### En este caso en concreto, usaría los datos originales, dado que los datos convertidos por PCA no son entendibles porque no representan una columna sino una estadística y lo hace más confuso, y porque me fije en que tardaba más el modelo en operar con los datos con dimensionalidad